In [36]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-123220")
exp = Experiment(workspace=ws, name="automl")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-123220
Azure region: southcentralus
Subscription id: 30d182b7-c8c4-421c-8fa0-d3037ecfe6d2
Resource group: aml-quickstarts-123220


In [37]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###


cluster_name = "amlcomp"
try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target {}.'.format(cluster_name))
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2",
                                                               max_nodes=4)

      # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

compute_target.wait_for_completion(show_output= True)
print("Azure Machine Learning Compute attached")

Creating a new compute target...
Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
Azure Machine Learning Compute attached


In [38]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
datapath="https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds=TabularDatasetFactory.from_delimited_files(path=datapath)

train_data, test_data = ds.random_split(percentage=0.33, seed=200)
label_column_name = 'y'
### YOUR CODE HERE ###

In [39]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric= 'accuracy',
    training_data= train_data,
    label_column_name= label_column_name,
    n_cross_validations=5,
    compute_target=compute_target,
    iterations=50,
    max_concurrent_iterations=10)

In [40]:
# Submit your automl run

### YOUR CODE HERE ###
automl_run=exp.submit(automl_config,show_output=True)

Running on remote.
Running on remote compute: amlcomp
Parent Run ID: AutoML_8663af1a-13bc-480b-b075-02738f1fccb3

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+----------------

In [41]:
# Wait for the remote run to complete
automl_run.wait_for_completion()

{'runId': 'AutoML_8663af1a-13bc-480b-b075-02738f1fccb3',
 'target': 'amlcomp',
 'status': 'Completed',
 'startTimeUtc': '2020-10-27T17:53:07.468434Z',
 'endTimeUtc': '2020-10-27T18:21:57.720711Z',
 'properties': {'num_iterations': '50',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'amlcomp',
  'AMLSettingsJsonString': '{"path":null,"name":"automl","subscription_id":"30d182b7-c8c4-421c-8fa0-d3037ecfe6d2","resource_group":"aml-quickstarts-123220","workspace_name":"quick-starts-ws-123220","region":"southcentralus","compute_target":"amlcomp","spark_service":null,"azure_service":"remote","_local_managed_run_id":null,"many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":50,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":5,"y_min":null,"y_max":null,"num_classes":n

In [42]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_run,fitted_model=automl_run.get_output()

print(best_run)
print(fitted_model)


Run(Experiment: automl,
Id: AutoML_8663af1a-13bc-480b-b075-02738f1fccb3_48,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                  min_samples_split=0.19736842105263158,
                                                                                                  min_weight_fraction_leaf=0.0,
       

In [43]:
best_model = best_run.register_model(model_name="automl", model_path="./")

In [44]:
#print best parameters and accuracy

print("Best Run Id: ", best_run.id)
print('\n Accuracy:', best_run.get_metrics()['accuracy'])

print("\n Parameters: ", best_run.get_details())

Best Run Id:  AutoML_8663af1a-13bc-480b-b075-02738f1fccb3_48

 Accuracy: 0.9131575469982481

 Parameters:  {'runId': 'AutoML_8663af1a-13bc-480b-b075-02738f1fccb3_48', 'target': 'amlcomp', 'status': 'Completed', 'startTimeUtc': '2020-10-27T18:19:40.474382Z', 'endTimeUtc': '2020-10-27T18:21:43.070215Z', 'properties': {'runTemplate': 'automl_child', 'pipeline_id': '__AutoML_Ensemble__', 'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'classification\',\'primary_metric\':\'accuracy\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'automl\',\'compute_target\':\'amlcomp\',\'subscription_id\':\'30d182b7-c8c4-421c-8fa0-d3037ecfe6d2\',\'region\':\'southcentralus\',\'spark_service\':None}","ensemble_run_id":"AutoML_8663af1a-13bc-480b-b075-02738f1fccb3_48","experiment_name":"automl","workspace_n

In [45]:
compute_target.delete()